# Creating NGO Proximity Searches
## Claude de Rijke-Thomas
## Sat 25th April 2020

In [1]:
import numpy as np
import scipy.spatial as spatial
import sklearn
import utm
from pyproj import Proj
import string
import random

In [2]:
class NGO:
    def __init__(self, ngo_id, ngo_name, ngo_loc_lon_lat, ngo_radius, ngo_speciality=None, ngo_loc_metres=None,
                 ngo_zone_number=None,ngo_zone_designator=None, ngo_number=None, ngo_url=None):
        """ An object representing the data for a single NGO"""
        self.id = ngo_id
        self.name = ngo_name
        self.loc_lon_lat = ngo_loc_lon_lat
        self.radius = ngo_radius #in m
        self.speciality = ngo_speciality
        self.loc_metres=ngo_loc_metres
        self.zone_designator=ngo_zone_designator
        self.zone_number=ngo_zone_number
        self.number=ngo_number
        self.url = ngo_url
    def add_location(self, ngo_loc_lon_lat):
        if self.loc_lon_lat is None:
            self.loc_lon_lat = []
        self.location.append(ngo_location)
    def add_speciality(self, ngo_speciality):
        if self.speciality is None:
            self.speciality = []
        self.speciality.append(ngo_speciality)
    def add_loc_metres(self, coords_tuple):
        if self.loc_metres is None:
            self.loc_metres = []
        if len(self.loc_metres)==0:
            self.loc_metres.extend([coords_tuple[0], coords_tuple[1]])
    def add_zone_number(self, ngo_zone_number):
        if self.zone_number is None:
            self.zone_number=ngo_zone_number
    def add_zone_designator(self, ngo_zone_designator):
        if self.zone_designator is None:
            self.zone_designator=(ngo_zone_designator)
    def add_number(self, ngo_number):
        if self.number is None:
            self.number=(ngo_number)
    def add_url(self, ngo_url):
        if self.url is None:
            self.url=(ngo_url)
class Human:
    def __init__(self, human_id, human_name, human_loc_lon_lat, human_speciality=None, human_loc_metres=None, 
                 human_zone_number=None,human_zone_designator=None):
        """ An object representing the data for a single Human"""
        self.id = human_id
        self.name = human_name
        self.loc_lon_lat = human_loc_lon_lat
        self.speciality = human_speciality
        self.loc_metres=human_loc_metres
        self.zone_designator=human_zone_designator
        self.zone_number=human_zone_number
    def add_location(self, human_loc_lon_lat):
        if self.loc_lon_lat is None:
            self.loc_lon_lat = []
        self.location.append(human_location)
    def add_speciality(self, human_speciality):
        if self.speciality is None:
            self.speciality = []
        self.speciality.append(human_speciality)
    def add_loc_metres(self, coords_tuple):
        if self.loc_metres is None:
            self.loc_metres = []
        if len(self.loc_metres)==0:
            self.loc_metres.extend([coords_tuple[0], coords_tuple[1]])
    def add_zone_number(self, human_zone_number):
        if self.zone_number is None:
            self.zone_number=human_zone_number
    def add_zone_designator(self, human_zone_designator):
        if self.zone_designator is None:
            self.zone_designator=(human_zone_designator)            
            

In [3]:
ngo_obj_arr = []
for i in range(10000):
    name=""
    for j in range(np.random.randint(3,12)):
        name+=random.choice(string.ascii_letters)
    ngo = NGO(np.random.bytes(10), name, [np.random.random()*180-90,np.random.random()*160-80], np.random.randint(1000000), "domestic abuse")
    ngo_obj_arr.append(ngo)

In [4]:
for ngo_obj in ngo_obj_arr:
    _,_,zone_number,zone_designator =utm.from_latlon(ngo_obj.loc_lon_lat[1], ngo_obj.loc_lon_lat[0])
    myProj = Proj("+proj=utm +zone="+str(zone_number)+str(zone_designator)+", +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    UTMx, UTMy = myProj(ngo_obj.loc_lon_lat[0], ngo_obj.loc_lon_lat[1])
    ngo_obj.add_loc_metres([UTMx,UTMy])
    ngo_obj.add_zone_number(zone_number)
    number="07"
    for j in range(9):
        number+=random.choice(string.digits)
    ngo_obj.add_zone_designator(zone_designator)
    ngo_obj.add_number(number)

In [5]:
human_obj = Human(np.random.bytes(10), "name", [np.random.random()*180-90,np.random.random()*160-80],"domestic abuse")
_,_,zone_number,zone_designator =utm.from_latlon(human_obj.loc_lon_lat[1], human_obj.loc_lon_lat[0])
myProj = Proj("+proj=utm +zone="+str(zone_number)+str(zone_designator)+", +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
UTMx, UTMy = myProj(human_obj.loc_lon_lat[0], human_obj.loc_lon_lat[1])
human_obj.add_loc_metres([UTMx,UTMy])
human_obj.add_zone_number(zone_number)
human_obj.add_zone_designator(zone_designator)

In [6]:
zonal_ngo_obj_arr = []
for ngo_obj in ngo_obj_arr:
    if ngo_obj.zone_number==human_obj.zone_number and ngo_obj.zone_designator==human_obj.zone_designator:
        zonal_ngo_obj_arr.append(ngo_obj)

In [7]:
print("The number of NGO's in the UTM Zone are:",len(zonal_ngo_obj_arr))

The number of NGO's in the UTM Zone are: 24


In [8]:
successful_ngo_obj_arr = []
for ngo_obj in zonal_ngo_obj_arr:
    if ngo_obj.radius > np.sqrt((human_obj.loc_metres[0]-ngo_obj.loc_metres[0])**2 + (human_obj.loc_metres[1]-ngo_obj.loc_metres[1])**2):
        successful_ngo_obj_arr.append(ngo_obj)
print("The number of NGO's available to the individual that fit the individual\'s requirements is: ",len(successful_ngo_obj_arr))
successful_ngo_names = [ngo_obj.name for ngo_obj in successful_ngo_obj_arr]
successful_ngo_numbers= [ngo_obj.number for ngo_obj in successful_ngo_obj_arr]
succ_name_n0_arr = [[ngo_obj.name,ngo_obj.number] for ngo_obj in successful_ngo_obj_arr]
print("The names and numbers of the NGO's available to the individual that fit the individual\'s requirements are: \n",succ_name_n0_arr)

The number of NGO's available to the individual that fit the individual's requirements is:  15
The names and numbers of the NGO's available to the individual that fit the individual's requirements are: 
 [['jDIC', '07454756835'], ['yPuHaqJUxW', '07500301704'], ['Oax', '07243310946'], ['mnrJnBUdfhw', '07047183829'], ['XOsWEAVS', '07349884686'], ['yxzz', '07045544123'], ['jRGCBVWOXlR', '07907756624'], ['BqSUgroCN', '07175142068'], ['Fgm', '07277500383'], ['EAWjuD', '07124552091'], ['NPGEKcwfqkP', '07648157657'], ['dcpKUSUwNrt', '07011617167'], ['SNyDUsGFg', '07950531082'], ['EwSosSVKpj', '07019066922'], ['JJDfip', '07219480399']]
